In [3]:
pip install ortools

     |████████████████████████████████| 14.0MB 316kB/s 
     |████████████████████████████████| 133kB 40.6MB/s 
     |████████████████████████████████| 1.0MB 43.2MB/s 
ERROR: tensorflow-metadata 0.27.0 has requirement absl-py<0.11,>=0.9, but you'll have absl-py 0.11.0 which is incompatible.
  Found existing installation: absl-py 0.10.0
    Uninstalling absl-py-0.10.0:
      Successfully uninstalled absl-py-0.10.0
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


# TSP

In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

In [2]:
def tsp(dist_function, locations):  # function takes loc 1 and loc 2 and returns dist between

  def create_dist_matrix(locations):
    matrix = np.zeros((len(locations), len(locations))).tolist()
    for l1 in range(len(matrix)):
      for l2 in range(len(matrix)):
        matrix[l1][l2] = dist_function(locations[l1], locations[l2])
    return matrix
        
  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  dist_matrix = create_dist_matrix(locations)
            
  manager = pywrapcp.RoutingIndexManager(len(dist_matrix), 1, 0)

  routing = pywrapcp.RoutingModel(manager)
  transit_callback_index = routing.RegisterTransitCallback(distance_callback)

  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

  solution = routing.SolveWithParameters(search_parameters)

  visited = []
  index = routing.Start(0)
  while not routing.IsEnd(index):
    visited.append(manager.IndexToNode(index))
    index = solution.Value(routing.NextVar(index))
  visited.append(manager.IndexToNode(index))
  
  return visited, solution.ObjectiveValue()  # route (each element is an index of locations), cost

In [3]:
matrix = [
  [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
  [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
  [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
  [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
  [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
  [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
  [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
  [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
  [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
  [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
  [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
  [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
  [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
]
dist_function = lambda a, b : matrix[a][b]
locations = list(range(len(matrix)))

In [8]:
tsp(dist_function, locations)

([0, 7, 2, 3, 4, 12, 6, 8, 1, 11, 10, 5, 9, 0], 7293)

In [5]:
def change_time_limit(time_limit):
  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
  search_parameters.time_limit.seconds = time_limit

In [6]:
change_time_limit(1)  # (makes no difference now bc small dataset)

In [9]:
tsp(dist_function, locations)

([0, 7, 2, 3, 4, 12, 6, 8, 1, 11, 10, 5, 9, 0], 7293)